In [13]:
from langchain_community.chat_models import ChatOpenAI
import logging
from typing import Type
from pydantic import BaseModel,Field
from langchain.tools import BaseTool

from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from duckduckgo_search import DDGS

from langchain.agents import initialize_agent,AgentType
from langchain_community.document_loaders import WebBaseLoader


llm=ChatOpenAI(temperature=0.1)

class WikipediaArgsSchema(BaseModel):
    query : str=Field(description="검색하고 싶은 내용을 입력하세요.")

class WikipediaTool(BaseTool):
    name: str = "WikipediaTool"
    description: str = "Use Wikipedia to retrieve relevant content."
    args_schema: Type[WikipediaArgsSchema] = WikipediaArgsSchema

    def _run(self, query: str):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)


class CustomDuckDuckGoSearchRunArgsSchema(BaseModel):
    query: str = Field(description="검색하고 싶은 내용을 입력하세요.")

class CustomDuckDuckGoSearchRun(BaseTool):
    name: str = "CustomDuckDuckGoSearchRun"
    description: str = "Use DuckDuckGo to retrieve relevant content."
    args_schema: Type[CustomDuckDuckGoSearchRunArgsSchema] = CustomDuckDuckGoSearchRunArgsSchema

    def _run(self, query: str):
        try:
            with DDGS() as ddgs:
                # max_results 제거한 검색
                results = ddgs.text(query)
                #url 추출출
                urls = [result['href'] for result in results if 'href' in result]
                if urls:
                    loader = WebBaseLoader(urls[0])  # 첫 번째 URL 로드
                    return loader.load()
                else:
                    return "No URLs found in DuckDuckGo search results."
        except TypeError as e:
            logging.error(f"TypeError in DuckDuckGoSearchRun: {e}")
            return "DuckDuckGoSearchRun encountered an error due to max_results."
        except Exception as e:
            logging.error(f"Unexpected error in DuckDuckGoSearchRun: {e}")
            return f"Unexpected error: {e}"




prompt = "Research about the XZ backdoor"
agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_error=True, #에러가 있으면 알아서 고쳐서 씀...미츘네
    tools=[
        WikipediaTool(),
        CustomDuckDuckGoSearchRun(),
    ]
)

response = agent.invoke(prompt)

if isinstance(response, dict):
    response = response.get("output", "")
elif not isinstance(response, str):
    response = str(response)

file_path = "file/result.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(response)

file_path



> Entering new AgentExecutor chain...

Invoking: `WikipediaTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.



Page: XZ Utils
Summary: XZ Utils (previ

'file/result.txt'